In [14]:
%run "0. INSTALLS & GLOBALS.ipynb"


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# Acquire List of Nobel Laureates

In [15]:
BASE_URL = "https://en.wikipedia.org/wiki/List_of_Nobel_laureates"

In [16]:
def get_etree_from(driver, URL):
    driver.get(URL)
    tree = etree.HTML(driver.page_source)
    
    return tree

In [17]:
tree = get_etree_from(DRIVER, BASE_URL)

In [28]:
## INITIALISE PANDAS DATAFRAME
columns = ["YEAR", 
           "PHYSICS", "PHYSICS_URL",
           "CHEMISTRY", "CHEMISTRY_URL",
           "MEDICINE", "MEDICINE_URL",
           "LITERATURE", "LITERATURE_URL",
           "PEACE", "PEACE_URL",
           "ECONOMY", "ECONOMY_URL"]
columns_length = len(columns)
df = pd.DataFrame(columns=columns)

## EXTRACT TABLE CONTAINING ALL NOBEL LAUREATES FROM ETREE
table = tree.xpath("//table[contains(@class, 'wikitable sortable sticky-header jquery-tablesorter')]")
table_body = table[0].xpath("./tbody/*") # i dont know why i cant join this xpath query to the one above im so confused

## PROCESS THE TABLE BODY! cleaning is here too
for index, item in enumerate(table_body):
    print(f"Processing row {index+1}...")
    processing_row = item.xpath("*")
    df_row = []

    # CLEAN VALUE TO INGEST
    # TODO: capture URL of authors
    for table_value in processing_row:
        text = ''.join(table_value.itertext())
        text = text.split("\n")[0]
        text = text.split("[")[0]
        href = table_value.xpath(".//@href")
            
        if text not in ["—", "None"]:
            df_row.append(text.strip())
        else:
            df_row.append("No Record")

        if href and not text.isnumeric():
            df_row.append(';'.join(href))
        elif text in ["—", "None"]:
            df_row.append("No Record")
    
    print(df_row)
        
    if (len(df_row) < columns_length): #something is fucked (probably world war 2)
        while (len(df_row) < columns_length):
            df_row.append("No Record")

    print(len(df_row))
    df.loc[-1] = df_row
    df.index = df.index + 1  # shifting index
    df = df.sort_index()  # sorting by index

Processing row 1...
['1901', 'Wilhelm Röntgen', '/wiki/Wilhelm_R%C3%B6ntgen', "Jacobus Henricus van 't Hoff", '/wiki/Jacobus_Henricus_van_%27t_Hoff', 'Emil von Behring', '/wiki/Emil_von_Behring', 'Sully Prudhomme', '/wiki/Sully_Prudhomme', 'Henry Dunant;Frédéric Passy', '/wiki/Henry_Dunant;/wiki/Fr%C3%A9d%C3%A9ric_Passy', 'No Record', 'No Record']
13
Processing row 2...
['1902', 'Hendrik Lorentz;Pieter Zeeman', '/wiki/Hendrik_Lorentz;/wiki/Pieter_Zeeman', 'Emil Fischer', '/wiki/Emil_Fischer', 'Ronald Ross', '/wiki/Ronald_Ross', 'Theodor Mommsen', '/wiki/Theodor_Mommsen', 'Élie Ducommun;Charles Albert Gobat', '/wiki/%C3%89lie_Ducommun;/wiki/Charles_Albert_Gobat']
13
Processing row 3...
['1903', 'Henri Becquerel;Pierre Curie;Marie Curie', '/wiki/Henri_Becquerel;/wiki/Pierre_Curie;/wiki/Marie_Curie', 'Svante Arrhenius', '/wiki/Svante_Arrhenius', 'Niels Ryberg Finsen', '/wiki/Niels_Ryberg_Finsen', 'Bjørnstjerne Bjørnson', '/wiki/Bj%C3%B8rnstjerne_Bj%C3%B8rnson', 'Randal Cremer', '/wiki/Ran

In [29]:
df = df.fillna("No Record")
df = df.replace("None", "No Record")
df[df["YEAR"] == "1935"]

,YEAR,PHYSICS,PHYSICS_URL,CHEMISTRY,CHEMISTRY_URL,MEDICINE,MEDICINE_URL,LITERATURE,LITERATURE_URL,PEACE,PEACE_URL,ECONOMY,ECONOMY_URL
89,1935,James Chadwick,/wiki/James_Chadwick,Frédéric Joliot-Curie;Irène Joliot-Curie,/wiki/Fr%C3%A9d%C3%A9ric_Joliot-Curie;/wiki/Ir...,Hans Spemann,/wiki/Hans_Spemann,No Record,No Record,Carl von Ossietzky,/wiki/Carl_von_Ossietzky,No Record,No Record


In [30]:
df.to_csv("outputs/Nobel_Laureates.csv")

# Further Cleaning

In [21]:
MASTER_DF = pd.read_csv("outputs/Nobel_Laureates.csv")
MASTER_DF

,Unnamed: 0,YEAR,PHYSICS,PHYSICS_URL,CHEMISTRY,CHEMISTRY_URL,MEDICINE,MEDICINE_URL,LITERATURE,LITERATURE_URL,PEACE,PEACE_URL,ECONOMY,ECONOMY_URL
0,0,2024,John Hopfield;Geoffrey Hinton,/wiki/John_Hopfield;/wiki/Geoffrey_Hinton,David Baker;Demis Hassabis;John M. Jumper,/wiki/David_Baker_(biochemist);/wiki/Demis_Has...,Victor Ambros;Gary Ruvkun,/wiki/Victor_Ambros;/wiki/Gary_Ruvkun,Han Kang,/wiki/Han_Kang,Nihon Hidankyo,/wiki/Nihon_Hidankyo,Daron Acemoglu; Simon Johnson;James A. Robinson,/wiki/Daron_Acemoglu;/wiki/Simon_Johnson_(econ...
1,1,2023,Pierre Agostini;Ferenc Krausz;Anne L'Huillier,/wiki/Pierre_Agostini;/wiki/Ferenc_Krausz;/wik...,Moungi Bawendi;Louis E. Brus;Alexey Ekimov,/wiki/Moungi_Bawendi;/wiki/Louis_E._Brus;/wiki...,Katalin Karikó;Drew Weissman,/wiki/Katalin_Karik%C3%B3;/wiki/Drew_Weissman,Jon Fosse,/wiki/Jon_Fosse,Narges Mohammadi,/wiki/Narges_Mohammadi,Claudia Goldin,/wiki/Claudia_Goldin
2,2,2022,Alain Aspect;John Clauser;Anton Zeilinger,/wiki/Alain_Aspect;/wiki/John_Clauser;/wiki/An...,Carolyn Bertozzi;Morten P. Meldal;Karl Barry S...,/wiki/Carolyn_Bertozzi;/wiki/Morten_P._Meldal;...,Svante Pääbo,/wiki/Svante_P%C3%A4%C3%A4bo,Annie Ernaux,/wiki/Annie_Ernaux,Ales Bialiatski;Memorial;Centre for Civil Libe...,/wiki/Ales_Bialiatski;/wiki/Memorial_(society)...,Ben Bernanke;Douglas Diamond;Philip H. Dybvig,/wiki/Ben_Bernanke;/wiki/Douglas_Diamond;/wiki...
3,3,2021,Giorgio Parisi;Klaus Hasselmann;Syukuro Manabe,/wiki/Giorgio_Parisi;/wiki/Klaus_Hasselmann;/w...,Benjamin List;David MacMillan,/wiki/Benjamin_List;/wiki/David_MacMillan,David Julius;Ardem Patapoutian,/wiki/David_Julius;/wiki/Ardem_Patapoutian,Abdulrazak Gurnah,/wiki/Abdulrazak_Gurnah,Maria Ressa;Dmitry Muratov,/wiki/Maria_Ressa;/wiki/Dmitry_Muratov,David Card;Joshua Angrist;Guido Imbens,/wiki/David_Card;/wiki/Joshua_Angrist;/wiki/Gu...
4,4,2020,Roger Penrose;Reinhard Genzel;Andrea M. Ghez,/wiki/Roger_Penrose;/wiki/Reinhard_Genzel;/wik...,Emmanuelle Charpentier;Jennifer Doudna,/wiki/Emmanuelle_Charpentier;/wiki/Jennifer_Do...,Harvey J. Alter;Michael Houghton;Charles M. Rice,/wiki/Harvey_J._Alter;/wiki/Michael_Houghton_(...,Louise Glück,/wiki/Louise_Gl%C3%BCck,World Food Programme,/wiki/World_Food_Programme,Paul Milgrom;Robert B. Wilson,/wiki/Paul_Milgrom;/wiki/Robert_B._Wilson
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,119,1905,Philipp Lenard,/wiki/Philipp_Lenard,Adolf von Baeyer,/wiki/Adolf_von_Baeyer,Robert Koch,/wiki/Robert_Koch,Henryk Sienkiewicz,/wiki/Henryk_Sienkiewicz,Bertha von Suttner,/wiki/Bertha_von_Suttner,No Record,No Record
120,120,1904,Lord Rayleigh,"/wiki/John_Strutt,_3rd_Baron_Rayleigh",William Ramsay,/wiki/William_Ramsay,Ivan Pavlov,/wiki/Ivan_Pavlov,Frédéric Mistral;José Echegaray,/wiki/Fr%C3%A9d%C3%A9ric_Mistral;/wiki/Jos%C3%...,Institut de Droit International,/wiki/Institut_de_Droit_International,No Record,No Record
121,121,1903,Henri Becquerel;Pierre Curie;Marie Curie,/wiki/Henri_Becquerel;/wiki/Pierre_Curie;/wiki...,Svante Arrhenius,/wiki/Svante_Arrhenius,Niels Ryberg Finsen,/wiki/Niels_Ryberg_Finsen,Bjørnstjerne Bjørnson,/wiki/Bj%C3%B8rnstjerne_Bj%C3%B8rnson,Randal Cremer,/wiki/Randal_Cremer,No Record,No Record
122,122,1902,Hendrik Lorentz;Pieter Zeeman,/wiki/Hendrik_Lorentz;/wiki/Pieter_Zeeman,Emil Fischer,/wiki/Emil_Fischer,Ronald Ross,/wiki/Ronald_Ross,Theodor Mommsen,/wiki/Theodor_Mommsen,Élie Ducommun;Charles Albert Gobat,/wiki/%C3%89lie_Ducommun;/wiki/Charles_Albert_...,No Record,No Record


## Separate Laureates by Field

In [22]:
NOBEL_PRIZE_FIELDS = ["Physics", "Chemistry", "Medicine", "Literature", "Peace", "Economy"]

In [23]:
def break_authors(MASTER_DF, field):
    df = MASTER_DF[["YEAR", field, field+"_URL"]]

    if field == "LITERATURE":
        df[['AUTHOR_1', 'AUTHOR_2']] = df[field].str.split(';', expand=True)
        df[["URL_1", "URL_2"]] = df[field+"_URL"].str.split(';', expand=True)
        df = df.drop(columns=[field, field+"_URL"])
        df = df.fillna("No Record")
        return df

    try:
        df[['AUTHOR_1', 'AUTHOR_2', 'AUTHOR_3']] = df[field].str.split(';', expand=True)
        df[["URL_1", "URL_2", "URL_3"]] = df[field+"_URL"].str.split(';', expand=True)
        df = df.drop(columns=[field, field+"_URL"])
        df = df.fillna("No Record")
        return df
    except Exception as err:
        print(f"{err}")
        return df

In [24]:
for field in NOBEL_PRIZE_FIELDS:
    print(f"Breaking authors for {field}.")
    df = break_authors(MASTER_DF, field.upper())
    df.to_csv(f"outputs/laureates_by_field/{field}.csv")

Breaking authors for Physics.
Breaking authors for Chemistry.
Breaking authors for Medicine.
Breaking authors for Literature.
Breaking authors for Peace.
Breaking authors for Economy.


C:\Users\lunet\AppData\Local\Temp\ipykernel_24144\2515540001.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['AUTHOR_1', 'AUTHOR_2', 'AUTHOR_3']] = df[field].str.split(';', expand=True)
C:\Users\lunet\AppData\Local\Temp\ipykernel_24144\2515540001.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['AUTHOR_1', 'AUTHOR_2', 'AUTHOR_3']] = df[field].str.split(';', expand=True)
C:\Users\lunet\AppData\Local\Temp\ipykernel_24144\2515540001.py:12: SettingWithCopyWarning: 
A value is trying to be set o